In [1]:
pip install python-mnist -user


Usage:   
  /opt/jupyterhub/anaconda/bin/python -m pip install [options] <requirement specifier> [package-index-options] ...
  /opt/jupyterhub/anaconda/bin/python -m pip install [options] -r <requirements file> [package-index-options] ...
  /opt/jupyterhub/anaconda/bin/python -m pip install [options] [-e] <vcs project url> ...
  /opt/jupyterhub/anaconda/bin/python -m pip install [options] [-e] <local project path> ...
  /opt/jupyterhub/anaconda/bin/python -m pip install [options] <archive url/path> ...

no such option: -u
Note: you may need to restart the kernel to use updated packages.


In [2]:
import math
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets
import matplotlib.pyplot as plt
from datetime import datetime
import numpy as numpy
import pandas as pd


#import data
data=pd.read_csv('/datc/parcel/notebooks/data/postnl/20201014_300_klanten.csv', parse_dates=[0])

#klant 153 eruit halen
cust_filter = data['cust_id'] == 'klant_165'
data = data.where(cust_filter).dropna()
data

,procesdag,cust_id,aantal_pakketten,aantal_pakketten_volgende_dag,validation_column
59,2015-01-02,klant_165,17.0,0.0,train
319,2015-01-03,klant_165,0.0,0.0,train
579,2015-01-04,klant_165,0.0,56.0,train
839,2015-01-05,klant_165,56.0,82.0,train
1099,2015-01-06,klant_165,82.0,390.0,train
...,...,...,...,...,...
504459,2020-04-25,klant_165,0.0,0.0,test
504719,2020-04-26,klant_165,0.0,0.0,test
504979,2020-04-27,klant_165,0.0,119.0,test
505239,2020-04-28,klant_165,119.0,70.0,test


In [3]:
# Define amount of packages yesterday
data['aantal_pakketten_gisteren'] = data['aantal_pakketten'].shift(1)
data['aantal_pakketten_gisteren'].fillna(0, inplace=True)

In [4]:
# Hyper-parameters 
input_size = 784
hidden_size = 500
num_classes = 10
num_epochs = 5
batch_size = 100
learning_rate = 0.001


# Set conditions
data = data.where(data['procesdag'].dt.year < 2020).dropna()
test_size = 365 # 3 months
validation_size = 365 # 3 months

# Split train, test and validation
df_train = data[:-test_size]
df_test = data[-test_size:]
df_valid = data[-validation_size:]

In [5]:
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size) 
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)  
    
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out